# Cancer Diagnosis Prediction 

## Feature Selection

Approach II: 
1. Train base boosting models - Light GBM, CatBoost, XGBoost. Plus one deep learning model - TabNet. 
2. Devide features by locations (`Region`, `Division`, `patient_state`) and apply Lasso Regression to each location separately to select important features
3. Apply a boosting model to each location. 

In [101]:
import pandas as pd

In [102]:
pd.set_option('display.max_columns', None)

In [103]:
train_base = pd.read_csv('train_1.csv')
test_base = pd.read_csv('test_1.csv')

In [104]:
train_base

,patient_id,patient_zip3,patient_age,population,density,age_median,age_under_10,age_10_to_19,age_20s,age_30s,age_40s,age_50s,age_60s,age_70s,age_over_80,male,female,married,divorced,never_married,widowed,family_size,family_dual_income,income_household_median,income_household_under_5,income_household_5_to_10,income_household_10_to_15,income_household_15_to_20,income_household_20_to_25,income_household_25_to_35,income_household_35_to_50,income_household_50_to_75,income_household_75_to_100,income_household_100_to_150,income_household_150_over,income_household_six_figure,income_individual_median,home_ownership,housing_units,home_value,rent_median,rent_burden,education_less_highschool,education_highschool,education_some_college,education_bachelors,education_graduate,education_college_or_above,education_stem_degree,labor_force_participation,unemployment_rate,self_employed,farmer,race_white,race_black,race_asian,race_native,race_pacific,race_other,race_multiple,hispanic,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02,DiagPeriodL90D,patient_race_Asian,patient_race_Black,patient_race_Hispanic,patient_race_Other,patient_race_White,payer_type_COMMERCIAL,payer_type_MEDICAID,payer_type_MEDICARE ADVANTAGE,Region_Midwest,Region_Northeast,Region_South,Region_West,Division_East North Central,Division_East South Central,Division_Middle Atlantic,Division_Mountain,Division_Pacific,Division_South Atlantic,Division_West North Central,Division_West South Central,patient_state_AK,patient_state_AL,patient_state_AR,patient_state_AZ,patient_state_CA,patient_state_CO,patient_state_DC,patient_state_DE,patient_state_FL,patient_state_GA,patient_state_HI,patient_state_IA,patient_state_ID,patient_state_IL,patient_state_IN,patient_state_KS,patient_state_KY,patient_state_LA,patient_state_MD,patient_state_MI,patient_state_MN,patient_state_MO,patient_state_MS,patient_state_MT,patient_state_NC,patient_state_ND,patient_state_NE,patient_state_NM,patient_state_NV,patient_state_NY,patient_state_OH,patient_state_OK,patient_state_OR,patient_state_PA,patient_state_SC,patient_state_SD,patient_state_TN,patient_state_TX,patient_state_UT,patient_state_VA,patient_state_WA,patient_state_WI,patient_state_WV,patient_state_WY,patient_gender_F,breast_cancer_diagnosis_code_1741,breast_cancer_diagnosis_code_1742,breast_cancer_diagnosis_code_1743,breast_cancer_diagnosis_code_1744,breast_cancer_diagnosis_code_1745,breast_cancer_diagnosis_code_1746,breast_cancer_diagnosis_code_1748,breast_cancer_diagnosis_code_1749,breast_cancer_diagnosis_code_1759,breast_cancer_diagnosis_code_19881,breast_cancer_diagnosis_code_C50,breast_cancer_diagnosis_code_C5001,breast_cancer_diagnosis_code_C50011,breast_cancer_diagnosis_code_C50012,breast_cancer_diagnosis_code_C50019,breast_cancer_diagnosis_code_C50021,breast_cancer_diagnosis_code_C5011,breast_cancer_diagnosis_code_C50111,breast_cancer_diagnosis_code_C50112,breast_cancer_diagnosis_code_C50119,breast_cancer_diagnosis_code_C5021,breast_cancer_diagnosis_code_C50211,breast_cancer_diagnosis_code_C50212,breast_cancer_diagnosis_code_C50219,breast_cancer_diagnosis_code_C5031,breast_cancer_diagnosis_code_C50311,breast_cancer_diagnosis_code_C50312,breast_cancer_diagnosis_code_C50319,breast_cancer_diagnosis_code_C5041,breast_cancer_diagnosis_code_C50411,breast_cancer_diagnosis_code_C50412,breast_cancer_diagnosis_code_C50419,breast_cancer_diagnosis_code_C50421,breast_cancer_diagnosis_code_C5051,breast_cancer_diagnosis_code_C50511,breast_cancer_diagnosis_code_C50512,breast_cancer_diagnosis_code_C50519,breast_cancer_diagnosis_code_C50611,breast_cancer_diagnosis_code_C50612,breast_cancer_diagnosis_code_C50619,breast_cancer_diagnosis_code_C5081,breast_cancer_diagnosis_code_C50811,breast_cancer_diagnosis_code_C50812,breast_cancer_diagnosis_code_C50819,breast_cancer_diagnosis_code_C509,breast_cancer_diagnosis_code_C5091,breast_cancer_diagnosis_code_C50911,breast_cancer_diagnosis_code_C50912,breast_cancer_diagnosis_code_C50919,b

In [105]:
target = train_base['DiagPeriodL90D']
train_base.drop(columns=target.name, axis=1, inplace=True) 

In [109]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
import numpy as np

def apply_lasso(df_slice, target, alpha=0.1,):
    # Splitting data into features (X) and target variable (y)
    y = target
    X = df_slice

    # Filter features based on correlation
    corr_matrix = X.corr().abs()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.8)]
    X_filtered = X.drop(columns=to_drop)

    # Standardize the Features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Fit LASSO regression
    lasso_model = Lasso(alpha=alpha)
    lasso_model.fit(X_scaled, y)


    # Extract important features
    lasso_coefficients = lasso_model.coef_
    important_features = X.columns[np.abs(lasso_coefficients) > 0]

    return list(important_features)

In [112]:
from boosting_models import apply_catboost, apply_lightgbm, apply_xgboost
from tabnet import apply_tabnet
from tqdm import tqdm

In [113]:
important_features = apply_lasso(train_base, target, alpha=0.006)
important_features

['patient_zip3',
 'patient_age',
 'income_household_75_to_100',
 'home_value',
 'education_less_highschool',
 'self_employed',
 'commute_time',
 'health_uninsured',
 'patient_race_Black',
 'patient_race_White',
 'payer_type_COMMERCIAL',
 'payer_type_MEDICAID',
 'patient_state_AL',
 'patient_state_CO',
 'patient_state_LA',
 'patient_state_MN',
 'patient_state_NV',
 'patient_state_VA',
 'breast_cancer_diagnosis_code_1741',
 'breast_cancer_diagnosis_code_1742',
 'breast_cancer_diagnosis_code_1743',
 'breast_cancer_diagnosis_code_1744',
 'breast_cancer_diagnosis_code_1745',
 'breast_cancer_diagnosis_code_1746',
 'breast_cancer_diagnosis_code_1748',
 'breast_cancer_diagnosis_code_1749',
 'breast_cancer_diagnosis_code_1759',
 'breast_cancer_diagnosis_code_19881',
 'breast_cancer_diagnosis_code_C5001',
 'breast_cancer_diagnosis_code_C50119',
 'breast_cancer_diagnosis_code_C50421',
 'breast_cancer_diagnosis_code_C50811',
 'metastatic_cancer_diagnosis_code_C773',
 'metastatic_cancer_diagnosis_c

In [114]:
best_catboost_model, best_cat_params, best_cat_score = apply_catboost(train_base[important_features], target, cv=5)
print("Best Parameters for base model:", best_cat_params)
print("Best ROC for base model:", best_cat_score)

Best Parameters for base model: {'depth': 6, 'iterations': 200, 'learning_rate': 0.05}
Best ROC for base model: 0.8056441585217543


**Best previous result for CatBoost**

- Best Parameters for base model: {'depth': 4, 'iterations': 200, 'learning_rate': 0.05}
- Best ROC for base model: 0.8019676819045394

In [93]:
best_gbm_model, best_gbm_params, best_gmb_score = apply_lightgbm(train_base[important_features], target, cv=5)
print("Best Parameters for base model:", best_gbm_params)
print("Best ROC for base model:", best_gmb_score)

Best Parameters for base model: {'learning_rate': 0.05, 'n_estimators': 50, 'num_leaves': 31}
Best ROC for base model: 0.8002775778631868


**Best previous result for light GBM**

- Best Parameters for base model: {'learning_rate': 0.01, 'n_estimators': 150, 'num_leaves': 31}
- Best ROC for base model: 0.7959317174144424

In [15]:
best_xgb_model, best_xgb_params, best_xgb_score = apply_xgboost(train_base[important_features], target, cv=5)
print("Best Parameters for base model:", best_xgb_params)
print("Best ROC for base model:", best_xgb_score)

Best Parameters for base model: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 150}
Best ROC for base model: 0.8001561991412572


**Best previous result for XGBoost**
- Best Parameters for base model: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 150}
- Best ROC for base model: 0.8001561991412572

In [94]:
best_tabnet_model, best_tabnet_params, best_tabnet_score = apply_tabnet(train_base[important_features], target, cv=5)
print("Best Parameters for base model:", best_tabnet_params)
print("Best ROC for base model:", best_tabnet_score)

C:\Users\Julia\AppData\Roaming\Python\Python310\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.75041 | val_0_auc: 0.7135  |  0:00:01s
epoch 1  | loss: 0.58128 | val_0_auc: 0.72773 |  0:00:02s
epoch 2  | loss: 0.53608 | val_0_auc: 0.74639 |  0:00:03s
epoch 3  | loss: 0.52254 | val_0_auc: 0.7538  |  0:00:04s
epoch 4  | loss: 0.51845 | val_0_auc: 0.75141 |  0:00:05s
epoch 5  | loss: 0.5143  | val_0_auc: 0.76124 |  0:00:06s
epoch 6  | loss: 0.51181 | val_0_auc: 0.75582 |  0:00:07s
epoch 7  | loss: 0.5028  | val_0_auc: 0.765   |  0:00:09s
epoch 8  | loss: 0.49913 | val_0_auc: 0.77345 |  0:00:10s
epoch 9  | loss: 0.49927 | val_0_auc: 0.77719 |  0:00:11s
epoch 10 | loss: 0.49722 | val_0_auc: 0.7791  |  0:00:12s
epoch 11 | loss: 0.4958  | val_0_auc: 0.78265 |  0:00:13s
epoch 12 | loss: 0.49228 | val_0_auc: 0.78815 |  0:00:14s
epoch 13 | loss: 0.49128 | val_0_auc: 0.7846  |  0:00:15s
epoch 14 | loss: 0.4884  | val_0_auc: 0.78728 |  0:00:17s
epoch 15 | loss: 0.48425 | val_0_auc: 0.79429 |  0:00:18s
epoch 16 | loss: 0.48244 | val_0_auc: 0.79749 |  0:00:19s
epoch 17 | los

C:\Users\Julia\AppData\Roaming\Python\Python310\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Best Parameters for base model: None
Best ROC for base model: 0.8136765948486919


**Best previous result for TabNet**

- Best ROC for base model: 0.8136765948486919

In [115]:
def test_and_pack(test, important_features, best_model, name):
    scaler = StandardScaler()
    scaled = scaler.fit_transform(test[important_features])

    pred_y = best_tabnet_model.predict(scaled)

    preds = pd.DataFrame(pred_y, columns=['DiagPeriodL90D'])
    preds.insert(0, 'patient_id', test['patient_id'])

    preds.to_csv(f'{name}_result.csv', index=False)

In [117]:
# test_and_pack(test_base, important_features, best_tabnet_model, 'tabnet')

In [84]:
test_and_pack(test_base, important_features, best_catboost_model, 'cat')

## Train Models on Locations

In [ ]:
train = pd.read_csv('train_3.csv')
test = pd.read_csv('test_3.csv')

In [ ]:
def group_by_location(train, test, location_type):
    
    drop_columns = {
        'patient_state': ['patient_gender', 'Division', 'Region'],
        'Region': ['patient_gender', 'Division', 'patient_state'],
        'Division': ['patient_gender', 'patient_state', 'Region']
    }

    columns_to_drop = drop_columns.get(location_type, [])

    train = train.drop(columns_to_drop, axis=1)
    test = test.drop(columns_to_drop, axis=1)


    # Get unique states
    unique_locations = train[location_type].unique()

    # Create a dictionary to store DataFrames for each state
    locations_dfs = {}
    target_dfs = {}

    # Iterate over unique states
    for location in unique_locations:
        # Filter DataFrame for the current state
        location_df = train[train[location_type] == location].copy()
        location_df.drop([location_type, 'patient_id', 'DiagPeriodL90D'], axis=1, inplace=True)

        target_df = train[train[location_type] == location]['DiagPeriodL90D'].copy()
        
        # Store the filtered DataFrame in the dictionary with state as the key
        locations_dfs[location] = location_df
        target_dfs[location] = target_df

    return locations_dfs, target_dfs, train, test



division_dfs, division_target_dfs, division_train, division_test = group_by_location(train, test, 'Division')
region_dfs, region_target_dfs, regioon_train, region_test = group_by_location(train, test, 'Region')
state_dfs, state_target_dfs, state_train, state_test = group_by_location(train, test, 'patient_state')

In [12]:
def apply_models_by_location(location_type, locations_dfs, target_dfs, train, test):

    unique_locations = train[location_type].unique()

    # Example usage for each slice
    for location in tqdm(unique_locations):  # tqdm wrapper to show progress bar
        important_features = apply_lasso(locations_dfs[location], target_dfs[location], alpha=0.1)
        best_catboost_model, best_params, best_score = apply_catboost(locations_dfs[location][important_features], target_dfs[location], cv=2)
        print("Best Parameters for", location, ":", best_params)
        print("Best ROC for", location, ":", best_score)

In [13]:
apply_models_by_location('Division', division_dfs, division_target_dfs, division_train, division_test)

 12%|█▎        | 1/8 [00:05<00:38,  5.50s/it]

Best Parameters for Pacific : {'depth': 4, 'iterations': 100, 'learning_rate': 0.05}
Best ROC for Pacific : 0.7084442052018601


 25%|██▌       | 2/8 [00:10<00:30,  5.11s/it]

Best Parameters for West South Central : {'depth': 4, 'iterations': 100, 'learning_rate': 0.05}
Best ROC for West South Central : 0.6829556070277161


 38%|███▊      | 3/8 [00:15<00:24,  4.95s/it]

...

Best Parameters for East South Central : {'depth': 4, 'iterations': 100, 'learning_rate': 0.05}
Best ROC for East South Central : 0.754126776001776


 88%|████████▊ | 7/8 [00:35<00:04,  4.96s/it]

Best Parameters for South Atlantic : {'depth': 4, 'iterations': 100, 'learning_rate': 0.05}
Best ROC for South Atlantic : 0.6569921755647057


100%|██████████| 8/8 [00:40<00:00,  5.03s/it]

Best Parameters for West North Central : {'depth': 4, 'iterations': 100, 'learning_rate': 0.05}
Best ROC for West North Central : 0.7563179052767561


In [14]:
apply_models_by_location('Region', region_dfs, region_target_dfs, regioon_train, region_test)

 25%|██▌       | 1/4 [00:06<00:18,  6.28s/it]

Best Parameters for West : {'depth': 4, 'iterations': 100, 'learning_rate': 0.05}
Best ROC for West : 0.7068186188330744


 50%|█████     | 2/4 [00:12<00:12,  6.17s/it]

Best Parameters for South : {'depth': 4, 'iterations': 100, 'learning_rate': 0.05}
Best ROC for South : 0.6899812250037188


 75%|███████▌  | 3/4 [00:17<00:05,  5.77s/it]

Best Parameters for Northeast : {'depth': 4, 'iterations': 100, 'learning_rate': 0.1}
Best ROC for Northeast : 0.7266128710787565


100%|██████████| 4/4 [00:23<00:00,  6.00s/it]

Best Parameters for Midwest : {'depth': 4, 'iterations': 100, 'learning_rate': 0.05}
Best ROC for Midwest : 0.7247134383437552


In [16]:
apply_models_by_location('patient_state', state_dfs, state_target_dfs, state_train, state_test)

  2%|▏         | 1/44 [00:04<03:20,  4.66s/it]

Best Parameters for CA : {'depth': 4, 'iterations': 100, 'learning_rate': 0.05}
Best ROC for CA : 0.6812905606797031


  5%|▍         | 2/44 [00:09<03:12,  4.57s/it]

Best Parameters for TX : {'depth': 4, 'iterations': 100, 'learning_rate': 0.05}
Best ROC for TX : 0.6711571358748778


  7%|▋         | 3/44 [00:14<03:16,  4.78s/it]

...

Best Parameters for DE : {'depth': 4, 'iterations': 100, 'learning_rate': 0.05}
Best ROC for DE : 0.64125


 98%|█████████▊| 43/44 [03:51<00:05,  5.39s/it]

Best Parameters for MS : {'depth': 4, 'iterations': 100, 'learning_rate': 0.05}
Best ROC for MS : 0.8175913760152891


100%|██████████| 44/44 [03:57<00:00,  5.39s/it]

Best Parameters for AK : {'depth': 4, 'iterations': 100, 'learning_rate': 0.05}
Best ROC for AK : 1.0
